This notebook builds the similarity matrix used in the construction of the network to which spectral clustering is applied. 

First, let's load some packages. 

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

import pandas as pd

All the comparison work is performed in __tmscorePreloaded.py__.  In particular, the primary structure of the two proteins to be compared are aligned, and then the tertiary structures corresponding to that alignment are scored using a Template Model Score algorithm.  

In [4]:
import tmscorePreloaded as tmscore

Below is an example of how two different serine protease structures are created so that they can be compared (scored) against each other. 

In [5]:
from Bio import PDB

parser = PDB.PDBParser(QUIET=True)
ppb = PDB.PPBuilder()

pdbFile = r'SP\pdb1a0h.ent'
chain ='B'
structure = parser.get_structure(chain, pdbFile)
structure = ppb.build_peptides(structure)[0]
s1 = structure

pdbFile = r'SP\pdb1a0j.ent'
chain ='A'
structure = parser.get_structure(chain, pdbFile)
structure = ppb.build_peptides(structure)[0]
s2 = structure

Once prepared, they are subsequently scored using the tmscore library. The actual score is __tm__. 

In [6]:
val = tmscore.TMscoring(s1, s2)

In [7]:
_, tm, rmsd = val.optimise()
tm

0.048358580688018432

The next cell loads the pdb file names and which of the chains in those files is to be used for the comparisons.  

In [8]:
SerProtPFam = pd.read_csv('pbd_chain_uniprotPF00089.csv')
SerProtPFam.head()

PDB CHAIN SP_PRIMARY  PFAM_ID
0  1a0h     B     P00735  PF00089
1  1a0j     A     P35033  PF00089
2  1a0l     A     P20231  PF00089
3  1a2c     H     P00734  PF00089
4  1a3b     H     P00734  PF00089

In the cell below we build all the structures that are to be compared against each other. 

In [ ]:
Structures = []
PDBs = []

for i in range(len(SerProtPFam)):
    pdb_i   = SerProtPFam.loc[i,'PDB']
    chain_i = SerProtPFam.loc[i,'CHAIN']
    FNi = r'SP\pdb%s.ent' % pdb_i
    
    try:
        structure = parser.get_structure(chain_i, FNi)
        structure = ppb.build_peptides(structure)[0]
    except:
        structure = []
 
    Structures.append(structure)
    PDBs.append(pdb_i)

The next cell simply shows you how big (memory-wise) that __Structures__ is. 

In [19]:
import sys
sys.getsizeof(Structures)

16560

The next cell shows you a summary of what is in the __DataFrame__ named __SerProtPFam__. 

In [6]:
SerProtPFam.describe()

PDB CHAIN SP_PRIMARY  PFAM_ID
count   1991  1991       1991     1991
unique  1991    21        128        1
top     1h1b     A     P00760  PF00089
freq       1  1209        447     1991

In [ ]:
import warnings 

N = len(SerProtPFam)
Sims = np.zeros( (N,N) )
with warnings.catch_warnings():
    warnings.simplefilter("ignore")


    for i in range(1200,1,-1):
        pdb_i   = PDBs[i]
        s_i = Structures[i]
        for j in range(i+1,N):
            pdb_j   = PDBs[j]
            s_j = Structures[j]

            try:
                val = tmscore.TMscoring(s_i, s_j)
                _, tm, rmsd = val.optimise()
                Sims[i,j] = Sims[i,j] = tm
            except:
                print("Error in comparing %s and %s" % (pdb_i,pdb_j))
        if(i % 10 == 0): 
            np.savetxt('Sims.txt',Sims)
            print("JUST COMPLETED STEP %s" % i)

__Sims__ is a very large matrix.  However, it is not too large for most modern computers, especially since you likely would want to run this on a cluster of some kind (would take a long, long time on a single processor).  

Nonetheless, in general we would have saved __Sims__ as a _sparse_ matrix.  We have in fact saved __Sims__ as a sparse matrix for those who just want to reconstruct the network and perform the clustering.  

In [3]:
from scipy import sparse

In [4]:
Simsparse = sparse.csr_matrix(Sims)
Simsparse

<1991x1991 sparse matrix of type '<class 'numpy.float64'>'
	with 48188 stored elements in Compressed Sparse Row format>

In [7]:
sparse.save_npz('SimilarityMatrix.npz',Simsparse)